In [2]:
import sys
pwd = "P:/dsv/DAMII/Practical Project/rl_stock_agents"
sys.path.append(pwd)

import prob_evaluate
import pandas as pd

score_array = [[96.43, 99.44, 83.63],
               [73.42, 81.34, 46.46],
               [72.30, 91.51, 72.31],
               [71.70, 66.16, 70.28],
               [71.67, 71.67, 71.67],
               [74.36, 77.08, 74.35],
               [70.63, 62.21, 44.91],
               [83.22, 80.63, 82.51],
               [98.22, 93.18, 93.21],
               [69.92, 99.90, 72.54]]


score_array_columns = ['NB', 'SVM', 'AB']
score_array_dataset_name = ['A','B','C','D','E','F','G','H','I','J']
df = pd.DataFrame(score_array, columns=score_array_columns)
df['dataset'] = score_array_dataset_name
df = df[['dataset','NB','SVM','AB']]

columns_for_rank = ['NB','SVM','AB']

test = prob_evaluate.generate_rank_array_from_dataframe(df,columns_for_rank,equal_rank_behav="mean")
print(test)
stat, critical_f_value, reject_null_hypo = prob_evaluate.iman_davenport_test(test,0.95)

print(f'f-score: {stat:.3f}\nf-critical: {critical_f_value:.3f}\nSignificant: {reject_null_hypo}')

results1 = prob_evaluate.nemenyi_test(test,0.95,columns_for_rank)
display(results1)

[[2. 2. 3. 1. 2. 2. 1. 1. 1. 3.]
 [1. 1. 1. 3. 2. 1. 2. 3. 3. 1.]
 [3. 3. 2. 2. 2. 3. 3. 2. 2. 2.]]
f-score: 1.227
f-critical: 3.555
Significant: False


[(('NB', 'SVM'), 0.0, 1.0478214542564015, False),
 (('NB', 'AB'), 0.5999999999999999, 1.0478214542564015, False),
 (('SVM', 'AB'), 0.5999999999999999, 1.0478214542564015, False)]